In [1]:
# imports
from enum import Enum
from googleapiclient.discovery import build
import google.datalab.storage as storage
from PIL import Image, ImageDraw
from StringIO import StringIO

In [171]:
#imports
from IPython.display import HTML
#from io import BytesIO
from base64 import b64encode
from PIL import Image

In [ ]:
# API key
APIKEY="IzaSyDvN7_zfln0Mrxx0MWWoJGeGbWqasQiinc"

In [2]:
############################################################################################### 
#
#  generate functionality
#
############################################################################################### 
class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5

In [103]:
def draw_boxes(image, blocks, color):
    """Draw a border around the image using the hints in the vector list."""
    draw = ImageDraw.Draw(image)

    for block in bounds:
      # positions might not be correctly stated
      one = block['vertices'][0]['x'], block['vertices'][0]['y']
      two = block['vertices'][1]['x'], block['vertices'][1]['y']
      three = block['vertices'][2]['x'], block['vertices'][2]['y']
      four = block['vertices'][3]['x'], block['vertices'][3]['y']
      draw.polygon([one, two, three, four], None, color)
    return image

In [83]:
def get_document_bounds(pages, feature):    
  bounds = []
  feature = FeatureType.BLOCK
  for page in pages:
    for block in page['blocks']:
      for paragraph in block['paragraphs']:
        for word in paragraph['words']:
          for symbol in word['symbols']:
            if (feature == FeatureType.SYMBOL):
              bounds.append(symbol['bounding_box'])
          if (feature == FeatureType.WORD):
            bounds.append(word['boundingBox'])
        if (feature == FeatureType.PARA):
            bounds.append(paragraph['boundingBox'])
      if (feature == FeatureType.BLOCK):
          bounds.append(block['boundingBox'])
    if (feature == FeatureType.PAGE):
        bounds.append(block['boundingBox'])
  return bounds

In [190]:
def render_doc_text(pages, img_data_in, img_data_out):
  out_image = Image.open( StringIO(img_data_in.read_stream()) )
  
  # iterate through its features
  bounds = get_document_bounds(pages, FeatureType.PAGE)
  draw_boxes(out_image, bounds, 'blue')

  bounds = get_document_bounds(pages, FeatureType.PARA)
  draw_boxes(out_image, bounds, 'red')

  bounds = get_document_bounds(pages, FeatureType.WORD)
  draw_boxes(out_image, bounds, 'yellow')
  
  my_file = StringIO()
  out_image.save('one.png', "PNG")
  #img_data_out.write_stream(b64encode(my_file.getvalue()).decode('utf-8')), 'text/plain')

In [90]:
def make_api_call(image):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
            'requests': [{
                    'image': {
                        'source': {
                            'imageUri': image
                        }
                    },
                    'features': [{
                        'type': 'DOCUMENT_TEXT_DETECTION',
                        'maxResults': 1,
                    }]
                }],
            })
    responses = request.execute(num_retries=3)
    return responses['responses'][0]['fullTextAnnotation']['pages']

In [114]:
# manipulate file names to create in and out file
def return_file_names(image):
  in_file = "gs://machine-learning-backend-storagebucket/" + image
  img_out = image.rsplit('/',1)[0] + "/output/" + image.rsplit('/',1)[1] + "_bound_box.png"
  img_in = image
  return in_file, img_out, img_in

In [ ]:
############################################################################################### 
#
#  separater
#
############################################################################################### 

In [ ]:
############################################################################################### 
#
#  Start document markup
#
############################################################################################### 
# select the storage bucket object you want
images = list()
my_bucket = storage.Bucket('machine-learning-backend-storagebucket')
for obj in my_bucket.objects():
  if obj.key.find("Oxygen") > -1 and obj.key.find(".txt") == -1:
    images.append(obj.key)

In [191]:
for image in images[1:2]:
  # create in and out names
  filein, img_out, img_in = return_file_names(image)
  
  # open the input image file
  img_data_in  = my_bucket.object(img_in)
  img_data_out = my_bucket.object(img_out)
  
  # call the API
  pages = make_api_call(filein)
  
  # render the document
  render_doc_text(pages, img_data_in, img_data_out)

In [170]:
print filein
print img_out
print img_in
print img_data_in
print img_data_out

gs://machine-learning-backend-storagebucket/sas_data/HLS/Oxygen_Case_1_02.png
sas_data/HLS/output/Oxygen_Case_1_02.png_bound_box.png
sas_data/HLS/Oxygen_Case_1_02.png
Google Cloud Storage Object gs://machine-learning-backend-storagebucket/sas_data/HLS/Oxygen_Case_1_02.png
Google Cloud Storage Object gs://machine-learning-backend-storagebucket/sas_data/HLS/output/Oxygen_Case_1_02.png_bound_box.png
